# Unsupervised Anomaly Detection: Schools Performance 2016-17
## Objective
The focus of this notebook/project is to investigate the use of **unsupervised anomaly detection** methods on **Schools Performance 2016-17** data. We are particularly interested in spotting anomalous pupil destination data. 

## Definitions
The following terms used in this project are defined below:
+ **Unsupervised data:** When your data is not labelled. *e.g. If you have income data but you do not know if this data is correct*
+ **Anomaly data:** Unusual and typically wrong observations/data points in your data.
+ **Unsupervised anomaly detection:** Spotting anomaly observations in your data without any labelled data to tell you whether your predictions are correct.

## Data
The data for this project comes from the [Schools Performance 2016-17](https://www.compare-school-performance.service.gov.uk/). The data is here very rich and detailed. For the sake of demonstration and simplification., we will only consider a subset of this information, defined below:
+ **URN** | Unique reference number for schools
+ **LEA** | Local authority, as a code
+ **ESTAB** | Establishment number
    - *Note that combining the LA and ESTAB fields to give LAESTAB will give us another unique identifer for schools, akin to our URN field*
+ **SCHNAME** | School name
+ **NFTYPE** | School type
    - AC = Sponsored academy
    - ACC = Academy converter – mainstream
    - AC1619 = Academy 16-19 sponsor led
    - ACC1619 = Academy 16-19 converter
    - ACCS = Academy converter - special school
    - ACS = Sponsored special academy, CTC=City technology college
    - CY = Community school
    - CYS = Community special school
    - F = Free school – mainstream
    - FESI = Further Education Sector Institution
    - FD = Foundation school
    - FDS = Foundation special school
    - FS = Free school – special
    - FSS = Studio school
    - FUTC = UTC (university technical college)
    - F1619 = Free school - 16-19
    - IND = Independent school
    - INDSPEC = Independent special school
    - MODFC = College funded by Ministry of Defence
    - NMSS = Non-maintained special school
    - VA = Voluntary aided school,
    - VC = Voluntary controlled school
+ **OVERALL_DESTPER** | Percentage of pupils who have been in a sustained education or employment destination for the first two terms, October 2015 to March 2016. 
+ **PTEALGRP2** | Percentage of eligible pupils with English-as-(an)-language (EAL)
+ **PTMOBN** | Percentage of pupils classified as non-mobile
+ **PTRWM_EXP** | Percentage of pupils reaching the expected stnadard in reading, writing and maths
+ **PSENELST** | Percentage of eligible pupils with special-education-needs (SEN)
+ **PTFSM6CLA1A_16** | Percentage of KS2 disdvantaged pupils
+ **PNUMFSM** | Percentage of pupils for free-school-meals

The anomalous pupil destination data that we are interested in spotting is therefore found within the **OVERALL_DESTPER** field. Note that this information is *provisional* so it is **unlabelled**.

## Set-up
Need to set-up our Jupyter notebook so that it has the required libraries and an environment is set-up so that when sharing this noteboook, others can use the same environment as we did here.

By default, the use of `!conda install` will install the package to the environment location that we are running the kernel in. This would be the environment from which we started Jupyter notebook from, but we can check this by looking at some of the system variables from the sys module.

In [2]:
import sys
sys.executable

'C:\\Users\\a_vis\\Anaconda3\\python.exe'

In [17]:
# Allows multiple outputs to show from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Show the two dataframes side-by-side by creating a function
from IPython.display import display_html
def display_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table', 'table style = "display:inline"'),raw = True)


In [3]:
# Use pip freeze to examine installed packages and versions within our Jupyter session
!pip freeze

alabaster==0.7.9
anaconda-clean==1.0
anaconda-client==1.6.3
anaconda-navigator==1.6.4
anaconda-project==0.6.0
argcomplete==1.0.0
astroid==1.4.7
astropy==1.2.1
Babel==2.3.4
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.5.1
bitarray==0.8.1
blaze==0.10.1
bokeh==0.12.2
boto==2.42.0
Bottleneck==1.1.0
cffi==1.7.0
chardet==3.0.4
chest==0.2.3
click==6.6
cloudpickle==0.2.1
clyent==1.2.2
colorama==0.3.7
comtypes==1.1.2
conda==4.5.0
conda-build==2.0.2
configobj==5.0.6
contextlib2==0.5.3
cryptography==1.5
cycler==0.10.0
Cython==0.24.1
cytoolz==0.8.0
dask==0.11.0
datashape==0.5.2
decorator==4.0.10
dill==0.2.5
docutils==0.12
dynd===c328ab7
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==2.0.6
Flask==0.11.1
Flask-Cors==2.1.2
gevent==1.1.2
greenlet==0.4.10
h5py==2.7.1
HeapDict==1.0.0
idna==2.1
imagesize==0.7.1
ipykernel==4.5.0
ipython==5.1.0
ipython-genutils==0.1.0
ipywidgets==5.2.2
itsdangerous==0.24
jdcal==1.2
jedi==0.9.0
Jinja2==2.8
jsonschema==2.5.1
jupyter==1.0.0
jupyter-client==4.4.0


You are using pip version 8.1.2, however version 9.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
# Install libraries not already above
!conda install seaborn -y

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\a_vis\Anaconda3

  added / updated specs: 
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    seaborn-0.8.1              |   py35hc73483e_0         338 KB
    openssl-1.0.2o             |       h8ea7d77_0         5.4 MB
    ------------------------------------------------------------
                                           Total:         5.7 MB

The following NEW packages will be INSTALLED:

    seaborn:         0.8.1-py35hc73483e_0            

The following packages will be UPDATED:

    ca-certificates: 2018.1.18-0          conda-forge --> 2018.03.07-0     
    openssl:         1.0.2n-vc14_0        conda-forge [vc14] --> 1.0.2o-h8ea7d77_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done



seaborn 0.8.1:            |   0% 
seaborn 0.8.1: 3          |   4% 
seaborn 0.8.1: #######5   |  75% 
seaborn 0.8.1: #########3 |  93% 
seaborn 0.8.1: ########## | 100% 

openssl 1.0.2o:            |   0% 
openssl 1.0.2o: 1          |   2% 
openssl 1.0.2o: 6          |   6% 
openssl 1.0.2o: #1         |  11% 
openssl 1.0.2o: #7         |  18% 
openssl 1.0.2o: ##2        |  23% 
openssl 1.0.2o: ##7        |  28% 
openssl 1.0.2o: ###3       |  34% 
openssl 1.0.2o: ###8       |  39% 
openssl 1.0.2o: ####4      |  45% 
openssl 1.0.2o: #####      |  50% 
openssl 1.0.2o: #####7     |  57% 
openssl 1.0.2o: ######3    |  63% 
openssl 1.0.2o: ######9    |  70% 
openssl 1.0.2o: #######5   |  76% 
openssl 1.0.2o: ########   |  80% 
openssl 1.0.2o: ########4  |  84% 
openssl 1.0.2o: ########8  |  88% 
openssl 1.0.2o: #########1 |  92% 
openssl 1.0.2o: #########4 |  95% 
openssl 1.0.2o: #########7 |  98% 
openssl 1.0.2o: ########## | 100% 


Having downloaded all our required libraries, and set-up our environment as we wanted, we will save this environment so others taking this Jupyter notebook can use the same environment as us in order to replicate our results, and not go through the same hassle of going through the parts above this comment block. They will download this environment within their own Anaconda Navigator.

Creating this environemnt, can upload it to my Anaconda Cloud account, which would allow other users to download and install it as a new environment within their own Anaconda Navigator.

In [1]:
# Export our environment, "NewEnv" and save it as "anomaly-detection.yml"
!conda env export -n NewEnv -f anomaly-detection.yml

# Data Load, Consolidation and Wrangling
In this section, we will load in our data, join our different data sources together, and manipulate our joine ddataframe so that it is in a format for us to use various **anomaly detection** algorithms on.

In [11]:
# Check working directory to ensure user notebook is easily transferable
import os
os.getcwd()

'C:\\Users\\a_vis\\Documents\\Data Science\\Projects\\Anomaly Detection\\local_master.git'

In [9]:
# Import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Display plots in Jupyter notebooks
% matplotlib inline

# Set random seed so we can reproduce our outputs
rng = np.random.RandomState(123)

# Load data
data_provisional_destination = pd.read_csv('Data 2016-2017/england_pupdestprovisional.csv', encoding = 'UTF-8')
data_final_ks2 = pd.read_csv('Data 2016-2017/england_ks2final.csv', encoding = 'UTF-8')

C:\Users\a_vis\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,2,3,4,12,14,16,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
# View data
display_side(data_provisional_destination.head(), data_final_ks2.head())

﻿RECTYPE 
 LEA 
 ESTAB 
 URN 
 ICLOSE 
 SCHNAME 
 NFTYPE 
 COHORT 
 OVERALL_DEST 
 APPREN 
 EMPLOYMENT 
 EDUCATION 
 FE 
 SCH_6TH 
 6TH_COL 
 OTHER_EDU 
 NOT_SUSTAINED 
 UNKNOWN 
 OVERALL_DESTPER 
 APPRENPER 
 EMPLOYMENTPER 
 EDUCATIONPER 
 FEPER 
 SCH_6THPER 
 6TH_COLPER 
 OTHER_EDUPER 
 NOT_SUSTAINEDPER 
 UNKNOWNPER 
 COHORT_DIS 
 OVERALL_DEST_DIS 
 APPREN_DIS 
 EMPLOYMENT_DIS 
 EDUCATION_DIS 
 FE_DIS 
 SCH_6TH_DIS 
 6TH_COL_DIS 
 OTHER_EDU_DIS 
 NOT_SUSTAINED_DIS 
 UNKNOWN_DIS 
 OVERALL_DESTPER_DIS 
 APPRENPER_DIS 
 EMPLOYMENTPER_DIS 
 EDUCATIONPER_DIS 
 FEPER_DIS 
 SCH_6THPER_DIS 
 6TH_COLPER_DIS 
 OTHER_EDUPER_DIS 
 NOT_SUSTAINEDPER_DIS 
 UNKNOWNPER_DIS 
 COHORT_NONDIS 
 OVERALL_DEST_NONDIS 
 APPREN_NONDIS 
 EMPLOYMENT_NONDIS 
 EDUCATION_NONDIS 
 FE_NONDIS 
 SCH_6TH_NONDIS 
 6TH_COL_NONDIS 
 OTHER_EDU_NONDIS 
 NOT_SUSTAINED_NONDIS 
 UNKNOWN_NONDIS 
 OVERALL_DESTPER_NONDIS 
 APPRENPER_NONDIS 
 EMPLOYMENTPER_NONDIS 
 EDUCATIONPER_NONDIS 
 FEPER_NONDIS 
 SCH_6THPER_NONDIS 
 6TH_COLPER_NONDIS 
 OTHER_EDUPER_NONDIS 
 NOT_SUSTAINEDPER_NONDIS 
 UNKNOWNPER_NONDIS 
 
 
 
 
 0 
 1 
 202.0 
 4104.0 
 100049.0 
 0.0 
 Haverstock School 
 CY 
 208 
 183 
 10 
 4 
 179 
 41 
 133 
 SUPP 
 SUPP 
 22 
 3 
 88% 
 5% 
 2% 
 86% 
 20% 
 64% 
 SUPP 
 SUPP 
 11% 
 1% 
 147 
 128 
 4 
 SUPP 
 SUPP 
 27 
 95 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 87% 
 3% 
 SUPP 
 SUPP 
 18% 
 65% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 61 
 55 
 6 
 SUPP 
 SUPP 
 14 
 38 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 90% 
 10% 
 SUPP 
 SUPP 
 23% 
 62% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 
 
 1 
 1 
 202.0 
 4166.0 
 100050.0 
 0.0 
 Parliament Hill School 
 CY 
 178 
 164 
 3 
 SUPP 
 SUPP 
 28 
 122 
 SUPP 
 SUPP 
 10 
 4 
 92% 
 2% 
 SUPP 
 SUPP 
 16% 
 69% 
 SUPP 
 SUPP 
 6% 
 2% 
 92 
 86 
 SUPP 
 SUPP 
 SUPP 
 19 
 60 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 93% 
 SUPP 
 SUPP 
 SUPP 
 21% 
 65% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 86 
 78 
 SUPP 
 SUPP 
 SUPP 
 9 
 62 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 91% 
 SUPP 
 SUPP 
 SUPP 
 10% 
 72% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 
 
 2 
 1 
 202.0 
 4196.0 
 100051.0 
 0.0 
 Regent High School 
 CY 
 127 
 115 
 0 
 SUPP 
 SUPP 
 44 
 65 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 91% 
 0% 
 SUPP 
 SUPP 
 35% 
 51% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 91 
 80 
 0 
 SUPP 
 SUPP 
 28 
 48 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 88% 
 0% 
 SUPP 
 SUPP 
 31% 
 53% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 36 
 35 
 0 
 SUPP 
 SUPP 
 16 
 17 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 97% 
 0% 
 SUPP 
 SUPP 
 44% 
 47% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 
 
 3 
 1 
 202.0 
 4275.0 
 100052.0 
 0.0 
 Hampstead School 
 CY 
 200 
 188 
 SUPP 
 SUPP 
 SUPP 
 44 
 135 
 SUPP 
 0 
 SUPP 
 SUPP 
 94% 
 SUPP 
 SUPP 
 SUPP 
 22% 
 68% 
 SUPP 
 0% 
 SUPP 
 SUPP 
 117 
 107 
 SUPP 
 SUPP 
 SUPP 
 27 
 75 
 SUPP 
 0 
 SUPP 
 SUPP 
 91% 
 SUPP 
 SUPP 
 SUPP 
 23% 
 64% 
 SUPP 
 0% 
 SUPP 
 SUPP 
 83 
 81 
 SUPP 
 SUPP 
 SUPP 
 17 
 60 
 SUPP 
 0 
 SUPP 
 SUPP 
 98% 
 SUPP 
 SUPP 
 SUPP 
 20% 
 72% 
 SUPP 
 0% 
 SUPP 
 SUPP 
 
 
 4 
 1 
 202.0 
 4285.0 
 100053.0 
 0.0 
 Acland Burghley School 
 CY 
 169 
 149 
 7 
 5 
 144 
 44 
 96 
 SUPP 
 SUPP 
 15 
 5 
 88% 
 4% 
 3% 
 85% 
 26% 
 57% 
 SUPP 
 SUPP 
 9% 
 3% 
 94 
 81 
 7 
 SUPP 
 SUPP 
 31 
 43 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 86% 
 7% 
 SUPP 
 SUPP 
 33% 
 46% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 75 
 68 
 0 
 SUPP 
 SUPP 
 13 
 53 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 91% 
 0% 
 SUPP 
 SUPP 
 17% 
 71% 
 SUPP 
 SUPP 
 SUPP 
 SUPP 
 
 
 
 
 
 
 ﻿RECTYPE 
 ALPHAIND 
 LEA 
 ESTAB 
 URN 
 SCHNAME 
 ADDRESS1 
 ADDRESS2 
 ADDRESS3 
 TOWN 
 PCODE 
 TELNUM 
 URN_AC 
 SCHNAME_AC 
 OPEN_AC 
 NFTYPE 
 ICLOSE 
 RELDENOM 
 AGERANGE 
 CONFEXAM 
 TAB15 
 TAB1618 
 TOTPUPS 
 TPUPYEAR 
 TELIG 
 BELIG 
 GELIG 
 PBELIG 
 PGELIG 
 TKS1AVERAGE 
 TKS1GROUP_L 
 PTKS1GROUP_L 
 TKS1GROUP_M 
 PTKS1GROUP_M 
 TKS1GROUP_H 
 PTKS1GROUP_H 
 TKS1GROUP_NA 
 PTKS1GROUP_NA 
 TFSM6CLA1A 
 PTFSM6CLA1A 
 TNotFSM6CLA1A 
 PTNotFSM6CLA1A 
 TEALGRP2 
 PTEALGRP2 
 TMOBN 
 PTMOBN 
 TSENELSE 
 PSENELSE 
 PTRWM_EXP 
 PTRWM_HIGH 
 READPROG 
 READPROG_LOWER 
 READPRO